In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install tfx

# Data Pipeline Components for Production ML

In this last graded programming exercise of the course, you will put together all the lessons we've covered so far to handle the first three steps of a production machine learning project - Data ingestion, Data Validation, and Data Transformation.

Specifically, you will build the production data pipeline by:

*   Performing feature selection
*   Ingesting the dataset
*   Generating the statistics of the dataset
*   Creating a schema as per the domain knowledge
*   Creating schema environments
*   Visualizing the dataset anomalies
*   Preprocessing, transforming and engineering your features
*   Tracking the provenance of your data pipeline using ML Metadata

Most of these will look familiar already so try your best to do the exercises by recall or browsing the documentation. If you get stuck however, you can review the lessons in class and the ungraded labs. 

Let's begin!

## Table of Contents

- [1 - Imports](#1)
- [2 - Load the Dataset](#2)
- [3 - Feature Selection](#4)
  - [Exercise 1 - Feature Selection](#ex-1)
- [4 - Data Pipeline](#4)
  - [4.1 - Setup the Interactive Context](#4-1)
  - [4.2 - Generating Examples](#4-2)
    - [Exercise 2 - ExampleGen](#ex-2)
  - [4.3 - Computing Statistics](#4-3)
    - [Exercise 3 - StatisticsGen](#ex-3)
  - [4.4 - Inferring the Schema](#4-4)
    - [Exercise 4 - SchemaGen](#ex-4)
  - [4.5 - Curating the Schema](#4-5)
    - [Exercise 5 - Curating the Schema](#ex-5)
  - [4.6 - Schema Environments](#4-6)
    - [Exercise 6 - Define the serving environment](#ex-6)
  - [4.7 - Generate new statistics using the updated schema](#4-7)
      - [Exercise 7 - ImporterNode](#ex-7)
      - [Exercise 8 - StatisticsGen with the new schema](#ex-8)
  - [4.8 - Check anomalies](#4-8)
      - [Exercise 9 - ExampleValidator](#ex-9)
  - [4.9 - Feature Engineering](#4-9)
      - [Exercise 10 - preprocessing function](#ex-10)
      - [Exercise 11 - Transform](#ex-11)
- [5 - ML Metadata](#5)
  - [5.1 - Accessing stored artifacts](#5-1)
  - [5.2 - Tracking artifacts](#5-2)
    - [Exercise 12 - Get parent artifacts](#ex-12)

<a name='1'></a>
## 1 - Imports

In [ ]:
import tensorflow as tf
import tfx

# TFX components
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.components import ImporterNode

# TFX libraries
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# For performing feature selection
from sklearn.feature_selection import SelectKBest, f_classif

# For feature visualization
import matplotlib.pyplot as plt 
import seaborn as sns

# Utilities
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict
from  tfx.proto import example_gen_pb2
from tfx.types import standard_artifacts
import os
import pprint
import tempfile
import pandas as pd
import numpy as np

# To ignore warnings from TF
tf.get_logger().setLevel('ERROR')

# For formatting print statements
pp = pprint.PrettyPrinter()

# Display versions of TF and TFX related packages
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow Data Validation version: {}'.format(tfdv.__version__))
print('TensorFlow Transform version: {}'.format(tft.__version__))

TensorFlow version: 2.6.2
TFX version: 1.4.0
TensorFlow Data Validation version: 1.4.0
TensorFlow Transform version: 1.4.0


<a name='2'></a>
## 2 - Load the dataset


In [ ]:
# # OPTIONAL: Just in case you want to restart the lab workspace *from scratch*, you
# # can uncomment and run this block to delete previously created files and
# # directories. 

# !rm -rf pipeline
# !rm -rf data

In [ ]:
TRAINING_DATA_DIR = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/train/real'
TRAINING_DATA = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/train/real/train.real.csv'

TRAINING_META_DIR = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/train/meta'
TRAINING_META = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/train/meta/train.meta.csv'



EVAL_DATA_DIR = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/real'
EVAL_DATA = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/real/train.real.csv'

EVAL_META_DIR = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/meta'
EVAL_META = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/meta/train.meta.csv'



TEST_DATA_DIR = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/real'
TEST_DATA = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/real/train.real.csv'

TEST_META_DIR = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/meta'
TEST_META = '/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/meta/train.meta.csv'


### Convert TSV to CSV

In [ ]:
import os
import glob
import base64

import pandas as pd
import tfrecorder


def convert_tsv_csv(dir):

  def labels_decode(x):
     return np.frombuffer(base64.b64decode(x['class_labels']), dtype=np.int64).reshape(x['num_boxes'])
  
  def box_decode(x):
    return np.frombuffer(base64.b64decode(x['boxes']), dtype=np.float32).reshape(x['num_boxes'], 4)

  def features_decode(x):
    return np.frombuffer(base64.b64decode(x['features']), dtype=np.float32).reshape(x['num_boxes'], 2048)


  meta_dir = os.path.join(dir, "meta/meta.csv")
  real_dir = os.path.join(dir, "real/real.csv")

  csv_files = glob.glob(os.path.join(dir, "*.tsv"))
  df = pd.concat([pd.read_csv(f, sep='\t') for f in csv_files], ignore_index = True)
  df['features'] = df.apply(features_decode, axis=1)
  print(df['features'].iloc[0])
  df['boxes'] = df.apply(box_decode, axis=1)
  df['class_labels'] = df.apply(labels_decode, axis=1)


  df.to_csv(real_dir, index=False)

ImportError: ignored

In [ ]:
dirs = ['/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/eval', '/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/test/testA', '/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/test/testB', '/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/train']

_ = [convert_tsv_csv(dir) for dir in dirs]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.         0.         0.03153366 ... 0.         0.         0.        ]]


KeyboardInterrupt: ignored

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/train/real/real.csv')
df.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,[[ 0. 3. 338. 207.]\n [239. 22. 333. 204.]],[[0. 0. 6.0378604 ... 0. ...,[28 28],treble popular reed,923271
1,101783080,80,61,1,[[ 5. 5. 75. 56.]],[[0. 0.01555114 1.4977295 ... 0. ...,[28],compatible ink cartridges,234031
2,100492797,750,750,1,[[ 43. 263. 658. 468.]],[[0. 0. 0.81766355 ... 0. ...,[7],check new look facial cream,199926
3,101708088,480,640,5,[[250. 421. 416. 635.]\n [252. 5. 429. 214.]...,[[0. 0. 0. ... 0. ...,[28 28 28 28 28],royal commemorative stamp,772725
4,100316842,800,800,4,[[138. 257. 716. 540.]\n [162. 667. 340. 795.]...,[[0. 0.01425867 0.02542738 ... 0. ...,[28 6 6 28],calendula baby shower gel,362576


In [ ]:
df.describe()

,product_id,image_h,image_w,num_boxes,query_id
count,1.000000e+04,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,1.015090e+08,759.558900,748.193500,3.432500,6.653151e+05
std,8.635320e+05,161.007375,158.297202,3.218925,3.977683e+05
min,1.000000e+08,72.000000,61.000000,1.000000,1.880000e+02
25%,1.007609e+08,750.000000,750.000000,1.000000,3.112230e+05
50%,1.015248e+08,800.000000,800.000000,2.000000,6.624170e+05
75%,1.022540e+08,800.000000,800.000000,4.000000,1.011362e+06
max,1.029998e+08,2309.000000,2200.000000,51.000000,1.355779e+06


In [ ]:
np.array_str(df['features'].iloc[0])

AttributeError: ignored

<a name='4'></a>
## 4 - Data Pipeline

With the selected subset of features prepared, you can now start building the data pipeline. This involves ingesting, validating, and transforming your data. You will be using the TFX components you've already encountered in the ungraded labs and you can look them up here in the [official documentation](https://www.tensorflow.org/tfx/api_docs/python/tfx/components).

<a name='4-1'></a>
### 4.1 - Setup the Interactive Context

As usual, you will first setup the Interactive Context so you can manually execute the pipeline components from the notebook. You will save the sqlite database in a pre-defined directory in your workspace. Please do not modify this path because you will need this in a later exercise involving ML Metadata.

In [ ]:
# Location of the pipeline metadata store
PIPELINE_DIR = './pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_DIR)

<a name='4-2'></a>
### 4.2 - Generating Examples

The first step in the pipeline is to ingest the data. Using [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen), you can convert raw data to TFRecords for faster computation in the later stages of the pipeline.

<a name='ex-2'></a>
#### Exercise 2: ExampleGen

Use `ExampleGen` to ingest the dataset we loaded earlier. Some things to note:

* The input is in CSV format so you will need to use the appropriate type of `ExampleGen` to handle it. 
* This function accepts a *directory* path to the training data and not the CSV file path itself. 

This will take a couple of minutes to run.

In [ ]:
# # NOTE: Uncomment and run this if you get an error saying there are different 
# # headers in the dataset. This is usually because of the notebook checkpoints saved in 
# # that folder.
# !rm -rf {TRAINING_DIR}/.ipynb_checkpoints
# !rm -rf {TRAINING_DIR_FSELECT}/.ipynb_checkpoints
# !rm -rf {SERVING_DIR}/.ipynb_checkpoints

In [ ]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [ ]:
# input_config_meta = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(name='train', pattern='/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/train/meta/*'),
#                 example_gen_pb2.Input.Split(name='eval', pattern='/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/eval/meta/*'),
#                 example_gen_pb2.Input.Split(name='testA', pattern='/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/test/testA/meta/*'),
#                 example_gen_pb2.Input.Split(name='testB', pattern='/gdrive/MyDrive/Study/Passau/Applied AI/Experiment/yasser_path2/data/test/testB/meta/*'),
#             ])



input_config_real = example_gen_pb2.Input(splits=[
                example_gen_pb2.Input.Split(name='train', pattern='/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/train/real/*'),
                example_gen_pb2.Input.Split(name='eval', pattern='/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/eval/real/*'),
                example_gen_pb2.Input.Split(name='testA', pattern='/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/test/testA/real/*'),
                example_gen_pb2.Input.Split(name='testB', pattern='/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/test/testB/real/*'),
            ])

In [ ]:
### START CODE HERE

# Instantiate ExampleGen with the input CSV dataset
data_example_gen = CsvExampleGen(input_base='/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data', input_config=input_config_real)

# Run the component using the InteractiveContext instance
context.run(data_example_gen)

### END CODE HERE

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 8
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/8"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"testA\", \"testB\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:3651308,xor_checksum:1638960608,sum_checksum:1638960608\nsplit:eval,num_files:1,total_bytes:6066014,xor_checksum:1638960512,sum_checksum:1638960512\nsplit:testA,num_files:1,total_bytes:11928088,xor_checksum:1638960554,sum_checksum:1638960554\nsplit:testB,num_files:1,total_bytes:12128249,xor_checksum:1638960595,sum_checksum:1638960595"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

<a name='4-3'></a>
### 4.3 - Computing Statistics

Next, you will compute the statistics of your data. This will allow you to observe and analyze characteristics of your data through visualizations provided by the integrated [FACETS](https://pair-code.github.io/facets/) library.

<a name='ex-3'></a>
#### Exercise 3: StatisticsGen

Use [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) to compute the statistics of the output examples of `ExampleGen`. 

In [ ]:
### START CODE HERE
stats_options = tfdv.StatsOptions(enable_semantic_domain_stats= True)
# Instantiate StatisticsGen with the ExampleGen ingested dataset
data_statistics_gen = StatisticsGen(examples=data_example_gen.outputs['examples'], stats_options=stats_options) 

# Run the component
context.run(data_statistics_gen)
### END CODE HERE

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 9
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/9"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"testA\", \"testB\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Display the results
context.show(data_statistics_gen.outputs['statistics'])

<a name='4-4'></a>
### 4.4 - Inferring the Schema

You will need to create a schema to validate incoming datasets during training and serving. Fortunately, TFX allows you to infer a first draft of this schema with the [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) component.

<a name='ex-4'></a>
#### Exercise 4: SchemaGen

Use `SchemaGen` to infer a schema based on the computed statistics of `StatisticsGen`.

In [ ]:
### START CODE HERE
# Instantiate SchemaGen with the output statistics from the StatisticsGen
data_schema_gen = SchemaGen(
    statistics=data_statistics_gen.outputs['statistics'],
    )
    
# Run the component
context.run(data_schema_gen)
### END CODE HERE

ExecutionResult(
    component_id: SchemaGen
    execution_id: 10
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 9
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/10"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Visualize the output
context.show(data_schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'boxes',BYTES,required,,-
'class_labels',BYTES,required,,-
'features',BYTES,required,,-
'image_h',INT,required,,-
'image_w',INT,required,,-
'num_boxes',INT,required,,-
'product_id',INT,required,,-
'query',BYTES,required,,-
'query_id',INT,required,,-


<a name='4-5'></a>
### 4.5 - Curating the schema

You can see that the inferred schema is able to capture the data types correctly and also able to show the expected values for the qualitative (i.e. string) data. You can still fine-tune this however. For instance, we have features where we expect a certain range.

You want to update your schema to take note of these so the pipeline can detect if invalid values are being fed to the model.

<a name='ex-5'></a>
#### Exercise 5: Curating the Schema

Use [TFDV](https://www.tensorflow.org/tfx/data_validation/get_started) to update the inferred schema to restrict a range of values to the features mentioned above.

In [ ]:
try:
    # Get the schema uri
    data_schema_uri = data_schema_gen.outputs['schema']._artifacts[0].uri
    
# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    schema_path = './pipeline/SchemaGen/data_schema'
    dir_id = os.listdir(schema_path)[0]
    data_schema_uri = f'{schema_path}/{dir_id}'

In [ ]:
# Get the schema pbtxt file from the SchemaGen output
data_schema = tfdv.load_schema_text(os.path.join(data_schema_uri, 'schema.pbtxt'))

In [ ]:
tfdv.display_schema(schema=data_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'boxes',BYTES,required,,-
'class_labels',BYTES,required,,-
'features',BYTES,required,,-
'image_h',INT,required,,-
'image_w',INT,required,,-
'num_boxes',INT,required,,-
'product_id',INT,required,,-
'query',BYTES,required,,-
'query_id',INT,required,,-


<a name='4-6'></a>
### 4.6 - Curating the schema

#### We can set the rang of our features vbased on the observed ranges from above statistics ^^

In [ ]:
### START CODE HERE ###

tfdv.set_domain(data_schema, 'query', schema_pb2.NaturalLanguageDomain())
tfdv.set_domain(data_schema, 'features', schema_pb2.ImageDomain())
tfdv.set_domain(data_schema, 'class_labels', schema_pb2.ImageDomain())
tfdv.set_domain(data_schema, 'boxes', schema_pb2.ImageDomain())

### END CODE HERE ###

tfdv.display_schema(schema=data_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'boxes',BYTES,required,,-
'class_labels',BYTES,required,,-
'features',BYTES,required,,-
'image_h',INT,required,,-
'image_w',INT,required,,-
'num_boxes',INT,required,,-
'product_id',INT,required,,-
'query',BYTES,required,,-
'query_id',INT,required,,-


You should now see the ranges you declared in the `Domain` column of the schema.

<a name='4-6'></a>
### 4.6 - Schema Environments

In supervised learning, we train the model to make predictions by feeding a set of features with its corresponding label. Thus, our training dataset will have both the input features and label, and the schema is configured to detect these. 

However, after training and you serve the model for inference, the incoming data will no longer have the label. This will present problems when validating the data using the current version of the schema. Let's demonstrate that in the following cells. You will simulate a serving dataset by getting subset of the training set and dropping the label column. Afterwards, you will validate this serving dataset using the schema you curated earlier.

In [ ]:
tf.data.experimental.CsvDataset

tensorflow.python.data.experimental.ops.readers.CsvDatasetV2

In [ ]:
# Declare paths to the serving data
DATA_DIR = '/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data'
TRAINING_DATA = '/gdrive/MyDrive/Study/Passau/Applied_AI/Experiment/yasser_path2/data/train/real/real.csv'
SERVING_DIR = f'{DATA_DIR}/serving'
SERVING_DATA = f'{SERVING_DIR}/serving_dataset.csv'

# Create the directory
!mkdir -p {SERVING_DIR}

In [ ]:
# Read a subset of the training dataset
serving_data = pd.read_csv(TRAINING_DATA, nrows=100)

# Save the modified dataset
serving_data.to_csv(SERVING_DATA, index=False)

# Delete unneeded variable from memory
del serving_data

In [ ]:
# Declare StatsOptions to use the curated schema
stats_options = tfdv.StatsOptions(schema=data_schema, infer_type_from_schema=True)

# Compute the statistics of the serving dataset
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=stats_options)

# Detect anomalies in the serving dataset
anomalies = tfdv.validate_statistics(serving_stats, schema=data_schema)

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

As expected, the missing column is flagged. To fix this, you need to configure the schema to detect when it's being used for training or for inference / serving. You can do this by setting [schema environments](https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic#schema_environments).

<a name='ex-6'></a>
#### Exercise 6: Define the serving environment

Complete the code below to ignore the `Cover_Type` feature when validating in the *SERVING* environment.

In [ ]:
schema.default_environment.append('TRAINING')

### START CODE HERE ###
# Hint: Create another default schema environment with name SERVING (pass in a string)
schema.default_environment.append('SERVING')

# Remove Cover_Type feature from SERVING using TFDV
# Hint: Pass in the strings with the name of the feature and environment 
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
### END CODE HERE ###

If done correctly, running the cell below should show *No Anomalies*.

In [ ]:
# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

We can now save this curated schema in a local directory so we can import it to our TFX pipeline.

In [ ]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = f'{PIPELINE_DIR}/updated_schema'

# Create the said directory
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(data_schema, schema_file)

As a sanity check, let's display the schema we just saved and verify that it contains the changes we introduced. It should still show the ranges in the `Domain` column and there should be two environments available.

In [ ]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'boxes',BYTES,required,,-
'class_labels',BYTES,required,,-
'features',BYTES,required,,-
'image_h',INT,required,,-
'image_w',INT,required,,-
'num_boxes',INT,required,,-
'product_id',INT,required,,-
'query',BYTES,required,,-
'query_id',INT,required,,-


In [ ]:
# The environment list should show `TRAINING` and `SERVING`.
new_schema.default_environment

[]

<a name='4-7'></a>
### 4.7 - Generate new statistics using the updated schema

You will now compute the statistics using the schema you just curated. Remember though that TFX components interact with each other by getting artifact information from the metadata store. So you first have to import the curated schema file into ML Metadata. You will do that by using an [ImporterNode](https://www.tensorflow.org/tfx/guide/statsgen#using_the_statsgen_component_with_a_schema) to create an artifact representing the curated schema.

<a name='ex-7'></a>
#### Exercise 7: ImporterNode

Complete the code below to create a `Schema` artifact that points to the curated schema directory. Pass in an `instance_name` as well and name it `import_user_schema`.

In [ ]:
### START CODE HERE ###

# Use an ImporterNode to put the curated schema to ML Metadata
user_schema_importer = ImporterNode(
    instance_name='import_user_schema',
    source_uri=UPDATED_SCHEMA_DIR,
    artifact_type=standard_artifacts.Schema
)

# Run the component
context.run(user_schema_importer, enable_cache=False)

### END CODE HERE ###

context.show(user_schema_importer.outputs['result'])

TypeError: ignored

With the artifact successfully created, you can now use `StatisticsGen` and pass in a `schema` parameter to use the curated schema.

<a name='ex-8'></a>
#### Exercise 8: Statistics with the new schema

Use `StatisticsGen` to compute the statistics with the schema you updated in the previous section. Remember to use the `stats_options` paremeter too to tell `StatisticsGen` to infer the data types from this new schema.

In [ ]:
### START CODE HERE ###
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = StatisticsGen(examples=example_gen.outputs['examples'], 
                                       stats_options=tfdv.StatsOptions(infer_type_from_schema=True),
                                      schema=user_schema_importer.outputs['result'])
    

# Run the component
context.run(statistics_gen_updated)
### END CODE HERE ###

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 5
        type_id: 7
        uri: "./pipeline/StatisticsGen/statistics/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [ ]:
context.show(statistics_gen_updated.outputs['statistics'])

The chart will look mostly the same from the previous runs but you can see that the `Cover Type` is now under the categorical features. That shows that `StatisticsGen` is indeed using the updated schema.

<a name='4-8'></a>
### 4.8 - Check anomalies

You will now check if the dataset has any anomalies with respect to the schema. You can do that easily with the [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component.

<a name='ex-9'></a>
#### Exercise 9: ExampleValidator

Check if there are any anomalies using `ExampleValidator`. You will need to pass in the updated statistics and schema from the previous sections.

In [ ]:
### START CODE HERE ###

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=user_schema_importer.outputs['result'])
    
    

# Run the component.
context.run(example_validator)

### END CODE HERE ###

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 7
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 6
        type_id: 12
        uri: "./pipeline/ExampleValidator/anomalies/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 12
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

<a name='4-10'></a>
### 4.10 - Feature engineering

You will now proceed to transforming your features to a form suitable for training a model. This can include several methods such as scaling and converting strings to vocabulary indices. It is important for these transformations to be consistent across your training data, and also for the serving data when the model is deployed for inference. TFX ensures this by generating a graph that will process incoming data both during training and inference.

Let's first declare the constants and utility function you will use for the exercise.

In [ ]:
# Set the constants module filename
_cover_constants_module_file = 'cover_constants.py'

In [ ]:
%%writefile {_cover_constants_module_file}

IMAGE = [
        "image_h",
        "image_w",
        "num_boxes",
        "boxes",
        "features",
        "class_labels"
    ]

QUERY = [
        "query",
        "query_id"
    ]

PRODUCT = [
        "product_id
    ]

LABEL_KEY = "relevancy"

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing cover_constants.py


Next you will define the `preprocessing_fn` to apply transformations to the features. 

<a name='ex-10'></a>
#### Exercise 10: Preprocessing function

Complete the module to transform your features. Refer to the code comments to get hints on what operations to perform.

Here are some links to the docs of the functions you will need to complete this function:

- [`tft.scale_by_min_max`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_by_min_max)
- [`tft.scale_to_0_1`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_0_1)
- [`tft.scale_to_z_score`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score)
- [`tft.compute_and_apply_vocabulary`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/compute_and_apply_vocabulary)
- [`tft.hash_strings`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/hash_strings)

In [ ]:
# Set the transform module filename
_cover_transform_module_file = 'cover_transform.py'

In [ ]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

_IMAGE = cover_constants.IMAGE
_QUERY = cover_constants.QUERY
_PRODUCT = cover_constants.PRODUCT
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    ### START CODE HERE ###
    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of min_max function
        # Hint: Use tft.scale_by_min_max by passing in the respective column
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of 0 to 1 function
        # Hint: tft.scale_to_0_1
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling to z score
        # Hint: tft.scale_to_z_score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using vocabulary available in column
        # Hint: Use tft.compute_and_apply_vocabulary
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform by hashing strings into buckets
        # Hint: Use tft.hash_strings with the param hash_buckets set to 10
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)
    
    ### END CODE HERE ###  

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict


Writing cover_transform.py


<a name='ex-11'></a>
#### Exercise 11: Transform

Use the [TFX Transform component](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Transform) to perform the transformations and generate the transformation graph. You will need to pass in the dataset examples, *curated* schema, and the module that contains the preprocessing function.

In [ ]:
### START CODE HERE ###
# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_cover_transform_module_file))
    
    
    
### END CODE HERE ###

# Run the component
context.run(transform, enable_cache=False)


ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "./pipeline/Transform/transform_graph/8"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 14
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 8
        type_id: 5
        uri: "./pipeline/Transform/transformed_examples/8"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 9
        type_id: 15
        uri: "./pipeline/Transform/updated_analyzer_cache/8"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 15
        name: "TransformCache"
        )]
        ))

Let's inspect a few examples of the transformed dataset to see if the transformations are done correctly.

In [ ]:
try:
    transform_uri = transform.outputs['transformed_examples'].get()[0].uri

# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    examples_path = './pipeline/Transform/transformed_examples'
    dir_id = os.listdir(examples_path)[0]
    transform_uri = f'{examples_path}/{dir_id}'

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# import helper function to get examples from the dataset
from util import get_records

# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['4']}},
                           'Elevation_xf': {'floatList': {'value': [-1.2982628]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.87007874]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9133858]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.875366]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.18468146]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [-1.1803539]}},
                           'Slope_xf': {'floatList': {'value': [-1.483387]}},
                           'Soil_Type_xf': {'int64List': {'value': ['4']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.22351421]}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['0']}}}}},
 {'featur

<a name='5'></a>
## 5 - ML Metadata

TFX uses [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) under the hood to keep records of artifacts that each component uses. This makes it easier to track how the pipeline is run so you can troubleshoot if needed or want to reproduce results.

In this final section of the assignment, you will demonstrate going through this metadata store to retrieve related artifacts. This skill is useful for when you want to recall which inputs are fed to a particular stage of the pipeline. For example, you can know where to locate the schema used to perform feature transformation, or you can determine which set of examples were used to train a model.

You will start by importing the relevant modules and setting up the connection to the metadata store. We have also provided some helper functions for displaying artifact information and you can review its code in the external `util.py` module in your lab workspace.

In [ ]:
# Import mlmd and utilities
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from util import display_types, display_artifacts, display_properties

# Get the connection config to connect to the metadata store
connection_config = context.metadata_connection_config

# Instantiate a MetadataStore instance with the connection config
store = mlmd.MetadataStore(connection_config)

# Declare the base directory where All TFX artifacts are stored
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

<a name='5-1'></a>
#### 5.1 -  Accessing stored artifacts

With the connection setup, you can now interact with the metadata store. For instance, you can retrieve all artifact types stored with the `get_artifact_types()` function. For reference, the API is documented [here](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore).

In [ ]:
# Get the artifact types
types = store.get_artifact_types()

# Display the results
display_types(types)

,id,name
0,5,Examples
1,7,ExampleStatistics
2,9,Schema
3,12,ExampleAnomalies
4,14,TransformGraph
5,15,TransformCache


You can also get a list of artifacts for a particular type to see if there are variations used in the pipeline. For example, you curated a schema in an earlier part of the assignment so this should appear in the records. Running the cell below should show at least two rows: one for the inferred schema, and another for the updated schema. If you ran this notebook before, then you might see more rows because of the different schema artifacts saved under the `./SchemaGen/schema` directory.

In [ ]:
# Retrieve the transform graph list
schema_list = store.get_artifacts_by_type('Schema')

# Display artifact properties from the results
display_artifacts(store, schema_list, base_dir)


,artifact id,type,uri
0,3,Schema,./SchemaGen/schema/3
1,4,Schema,./updated_schema


Moreover, you can also get the properties of a particular artifact. TFX declares some properties automatically for each of its components. You will most likely see `name`, `state` and `producer_component` for each artifact type. Additional properties are added where appropriate. For example, a `split_names` property is added in `ExampleStatistics` artifacts to indicate which splits the statistics are generated for.

In [ ]:
# Get the latest TransformGraph artifact
statistics_artifact = store.get_artifacts_by_type('ExampleStatistics')[-1]

# Display the properties of the retrieved artifact
display_properties(store, statistics_artifact)

,property,value
0,split_names,"[""train"", ""eval""]"
1,state,published
2,producer_component,StatisticsGen
3,name,statistics


<a name='5-2'></a>
#### 5.2 - Tracking artifacts

For this final exercise, you will build a function to return the parent artifacts of a given one. For example, this should be able to list the artifacts that were used to generate a particular `TransformGraph` instance. 

<a name='ex-12'></a>
##### Exercise 12: Get parent artifacts

Complete the code below to track the inputs of a particular artifact.

Tips:

* You may find [get_events_by_artifact_ids()](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore#get_events_by_artifact_ids) and [get_events_by_execution_ids()](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore#get_executions_by_id) useful here. 

* Some of the methods of the MetadataStore class (such as the two given above) only accepts iterables so remember to convert to a list (or set) if you only have an int (e.g. pass `[x]` instead of `x`).



In [ ]:
def get_parent_artifacts(store, artifact):

    ### START CODE HERE ###
    
    # Get the artifact id of the input artifact
    artifact_id = artifact.id
    
    # Get events associated with the artifact id
    artifact_id_events = store.get_events_by_artifact_ids([artifact_id])
    
    # From the `artifact_id_events`, get the execution ids of OUTPUT events.
    # Cast to a set to remove duplicates if any.
    execution_id = set( 
        event.execution_id
        for event in artifact_id_events # @REPLACE
        if event.type == metadata_store_pb2.Event.OUTPUT # @REPLACE
    )
    
    # Get the events associated with the execution_id
    execution_id_events = store.get_events_by_execution_ids(execution_id)

    # From execution_id_events, get the artifact ids of INPUT events.
    # Cast to a set to remove duplicates if any.
    parent_artifact_ids = set( 
        event.artifact_id
        for event in execution_id_events
        if event.type == metadata_store_pb2.Event.INPUT
    )
    
    # Get the list of artifacts associated with the parent_artifact_ids
    parent_artifact_list = [artifact for artifact in store.get_artifacts_by_id(parent_artifact_ids)]

    ### END CODE HERE ###
    
    return parent_artifact_list

In [ ]:
# Get an artifact instance from the metadata store
artifact_instance = store.get_artifacts_by_type('TransformGraph')[0]

# Retrieve the parent artifacts of the instance
parent_artifacts = get_parent_artifacts(store, artifact_instance)

# Display the results
display_artifacts(store, parent_artifacts, base_dir)

,artifact id,type,uri
0,1,Examples,./CsvExampleGen/examples/1
1,3,Schema,./SchemaGen/schema/3


**Expected Output:**

*Note: The ID numbers may differ.*

| artifact id | type | uri |
| ----------- | ---- | --- |
| 1	| Examples | ./CsvExampleGen/examples/1 |
| 4	| Schema | ./updated_schema |

**Congratulations!** You have now completed the assignment for this week. You've demonstrated your skills in selecting features, performing a data pipeline, and retrieving information from the metadata store. Having the ability to put these all together will be critical when working with production grade machine learning projects. For next week, you will work on more data types and see how these can be prepared in an ML pipeline. **Keep it up!**